In [ ]:
import numpy as np
import tensorflow as tf
from utils import load_w2v, get_test_idx, load_test_img, load_test_script
from os.path import isfile, join
from skimage import io
from skimage.transform import resize
from vgg_with_rnn import fused_model, rnn_encoder
import matplotlib.pyplot as plt


In [ ]:
w2v_path = './data/GoogleNews-vectors-negative300.bin'
w2v_model = load_w2v(w2v_path)

In [ ]:
npzfile = np.load('./data/preprocessed4QGN.npz')

img_idx = npzfile['arr_0']
question_embeds = npzfile['arr_2']
reverse_label = npzfile['arr_6']

print 'End loading MetaData'

In [ ]:
num_data = img_idx.shape[0]
dim_sentence = question_embeds.shape[1]
max_step_sentence = question_embeds.shape[2]
num_label = len(reverse_label)

idx_test, idx_train = get_test_idx(img_idx, random_seed=1, divide_rate=20, num_data=num_data)

In [ ]:
heatmap_resize = 64
img_resize = 64

curr_test_img_idx = 127
img_path = './data/train_img'
curr_test_img = io.imread('%s/%04d.jpg' %(img_path, curr_test_img_idx))
curr_test_img = resize(curr_test_img, [img_resize, img_resize], preserve_range=True)
curr_test_img = curr_test_img / 255.0

plt.imshow(curr_test_img)

test_question_path = './data/question_ambiguous'

npzfile = np.load('%s/%04d.npz' %(test_question_path, curr_test_img_idx))
curr_question = npzfile['arr_0']
curr_question = curr_question.item()

In [ ]:
curr_keys = curr_question.keys()
print curr_keys

In [ ]:
key_idx = 2

print curr_keys[key_idx][0:-1]
curr_esti = curr_question[curr_keys[key_idx]]['mean']
curr_uncertainty = curr_question[curr_keys[key_idx]]['var']

curr_esti = resize(curr_esti, [heatmap_resize, heatmap_resize], preserve_range=True)
curr_uncertainty = resize(curr_uncertainty, [heatmap_resize, heatmap_resize], preserve_range=True)

curr_combine_map = np.zeros((1, img_resize, img_resize, 4))
curr_upperbound = curr_esti + 2 * np.sqrt(curr_uncertainty)
curr_combine_map[0, :, :, 0:3] = curr_test_img
curr_combine_map[0, :, :, 3] = curr_upperbound

curr_embed_input, curr_seq_len = load_test_script(curr_keys[key_idx][0:-1], w2v_model, 
                                                  dim_sentence, max_step_sentence) 

In [ ]:
ph_combine = tf.placeholder(dtype=tf.float32, shape=[None, img_resize, img_resize, 4])
ph_sen = tf.placeholder(dtype=tf.float32, shape=[None, dim_sentence, max_step_sentence])
ph_sen_len = tf.placeholder(tf.int32, [None, 1])
ph_answer_label = tf.placeholder(dtype=tf.float32, shape=[None, num_label])
ph_keepprob = tf.placeholder(tf.float32)

my_rnn_encoder = rnn_encoder(dim_rnn_cell = dim_rnn_cell,
                             dim_sentence = dim_sentence,
                             max_step_sentence = max_step_sentence)
rnn_feature = my_rnn_encoder.encode(ph_sen, ph_sen_len)

my_fused_model = fused_model()

answer_logits = my_fused_model.vgg_with_rnn(rnn_feature = rnn_feature,
                                           init_image = ph_combine,
                                           keep_prob = ph_keepprob,
                                           n_classes = num_label)

In [ ]:
init = tf.global_variables_initializer()

saver = tf.train.Saver(var_list=tf.trainable_variables())

config = tf.ConfigProto()
config.allow_soft_placement = True
config.gpu_options.allow_growth = True

restore_path = './QGN_model/model.ckpt-1500'

print 'Now ready to start the session.'

with tf.Session(config=config) as sess:
    sess.run(init)
    saver.restore(sess, restore_path)
    
    test_feed_dict={ph_combine: curr_combine_map, ph_sen: curr_embed_input,
                    ph_sen_len: curr_seq_len, ph_keepprob: 1.0}
    curr_answer = sess.run(tf.nn.softmax(answer_logits), feed_dict=test_feed_dict)


In [ ]:
sorted_idx = np.argsort(-curr_answer)
print curr_answer
plt.figure(1)
plt.imshow(curr_test_img)
plt.suptitle(curr_keys[key_idx])

plt.figure(2)
plt.imshow(curr_esti)
plt.figure(3)
plt.imshow(curr_uncertainty)

print reverse_label[sorted_idx]
